In [1]:
# Import all the needed libraries
import pyterrier as pt
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import algos
import os
if not pt.started():
    pt.init()

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
# Load the dataset
dataset = pt.datasets.get_dataset("trec-deep-learning-passages")

In [3]:
index_ref = pt.IndexRef.of(os.getcwd() + "./passage_index/data.properties")

In [4]:
index = pt.IndexFactory.of(index_ref)

00:41:23.291 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1,9 GiB of memory would be required.


In [7]:
qe = pt.BatchRetrieve(index_ref, wmodel="BM25", controls={"qe" : "on"})

00:42:41.527 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1,9 GiB of memory would be required.


In [11]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [8]:
print(index.getCollectionStatistics().toString())

Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



In [13]:
pt.Experiment(
  [bm25, qe],
  dataset.get_topics("test-2019"), 
  dataset.get_qrels("test-2019"),
  eval_metrics=["ndcg", "map", "recip_rank"], 
  # perquery = "true",
  names=["BM25", "QE"],
  baseline=0,
  round=3,
)

13:21:50.297 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


,name,map,recip_rank,ndcg,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,ndcg +,ndcg -,ndcg p-value
0,BM25,0.37,0.795,0.593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,QE,0.40,0.788,0.611,22.0,21.0,0.02194,4.0,6.0,0.758362,23.0,20.0,0.118166
